# The Battle of Neighborhoods

In [1]:
import pandas as pd # Data Analysis
import numpy as np # Handles Data in a Vectorized Manner
import requests #handles requests
import folium # Plotting
from geopy.geocoders import Nominatim #address to Latitude and Longitude
from bs4 import BeautifulSoup


print('Modules Imported!')

Modules Imported!


In [2]:
# Web Sources for scraping

NYC_url = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'
LA_url = 'http://www.laalmanac.com/communications/cm02_communities.php'
CHI_url = 'https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php'

## Web Scraping and setting up DataFrames

New York City

In [3]:
source1 = requests.get(NYC_url).text
soup1 = BeautifulSoup(source1,'lxml')
table1 = soup1.find('table')

readme_html1 = pd.read_html(str(table1))
NYC_df = pd.DataFrame(readme_html1[0])
print(NYC_df.shape, '\n', NYC_df.head())

(42, 3) 
   Borough                Neighborhood                   ZIP Codes
0   Bronx               Central Bronx         10453, 10457, 10460
1   Bronx      Bronx Park and Fordham         10458, 10467, 10468
2   Bronx  High Bridge and Morrisania         10451, 10452, 10456
3   Bronx  Hunts Point and Mott Haven  10454, 10455, 10459, 10474
4   Bronx   Kingsbridge and Riverdale                10463, 10471


Los Angeles 

In [4]:
source2 = requests.get(LA_url).text
soup2 = BeautifulSoup(source2, 'lxml')
table2 = soup2.find('table')

readme_html2 = pd.read_html(str(table2))
LA_df = pd.DataFrame(readme_html2[0])
print(LA_df.shape, '\n', LA_df.head())

(643, 2) 
             City/Community   Zip Code(s)
0                    Acton         93510
1             Agoura Hills         91301
2  Agoura Hills (PO Boxes)         91376
3               Agua Dulce         91390
4                 Alhambra  91801, 91803


Chicago

In [5]:
source3 = requests.get(CHI_url).text
soup3 = BeautifulSoup(source3, 'lxml')
table3 = soup3.find('table')

readme_html3 = pd.read_html(str(table3))
CHI_df = pd.DataFrame(readme_html3[0])
print(CHI_df.shape, '\n', CHI_df.head())



(199, 2) 
                     0             1
0            Downtown      Zip Code
1  Cathedral District         60611
2     Central Station         60605
3       Dearborn Park         60605
4          Gold Coast  60610, 60611
